In [ ]:
            
            shadowcliquesetting = np.zeros(self.num_qubits,dtype=int)
            for idx in reversed(order):
                o = self.obs[idx]
                if verbose:
                    print("Checking",o)
                if hit_by(o,shadowcliquesetting):
                    non_id = o!=0
                    shadowcliquesetting[non_id] = o[non_id]
                if verbose:
                    print("p =",setting)
                if np.min(shadowcliquesetting) > 0:
                    break
            if verbose:
                print("Checking list of observables.")
            #cliques_with_epsilon = []
            delta = 0.02
            removedcliques = 0
            completecliques = 0
            valid_settings = []
            valid_cliques = []
            #start
            if center_node not in self.processed_center_node:  #gggggggg
                self.processed_center_node.append(center_node)
                hit_list = []
                hit_cliques = []
                non_id = first_obs != 0
                globalsetting[non_id] = first_obs[non_id]
                for o in self.obs:
                    if hit_by(o, globalsetting):
                        hit_list.append(o)
                if verbose:
                    print("First observable (setting):", first_obs)
                    print("Other observables hit by it:")
                    for ob in hit_list:
                        print(ob)
                if not hit_list:
                    raise RuntimeError("No hit list found.")
                hit_graph = build_hit_graph(hit_list)
                if not hit_graph:
                    raise RuntimeError("No hit graph found.")
                hit_cliques = find_cliques5(hit_graph)
                #print("length of hit cliques is", len(hit_cliques))
                self.clique_cache[center_node] = hit_cliques
                if not hit_cliques:
                    hit_cliques=[[center_node]]
                self.clean_setting_cache[center_node] = []
                self.clean_clique_cache[center_node] = []
                for clique in hit_cliques:
                    setting_candidate = np.zeros(self.num_qubits, dtype=int)
                    for o in clique:
                        o_arr = np.array(o)
                        if hit_by(o_arr, setting_candidate):
                            non_id = o_arr != 0
                            setting_candidate[non_id] = o_arr[non_id]
                        if np.min(setting_candidate) > 0:
                            break
                    """if np.min(setting_candidate) == 0:
                        for idx in reversed(order):
                            o = self.obs[idx]
                            if verbose:
                                print("Checking",o)
                            if hit_by(o,setting_candidate):
                                non_id = o!=0
                                setting_candidate[non_id] = o[non_id]
                            if verbose:
                                print("p =",setting_candidate)
                            if np.min(setting_candidate) > 0:
                                break"""
                    self.clean_clique_cache[center_node].append(clique)
                    if not any((existing == setting_candidate).all() for existing in self.clean_setting_cache[center_node]):
                        self.clean_setting_cache[center_node].append(setting_candidate)
            else:
                print("I already found cliques for this node")
                """else:
                    completecliques += 1
                    self.clean_clique_cache[center_node].append(clique)
                    if not any((existing == setting_candidate).all() for existing in self.clean_setting_cache[center_node]):
                        self.clean_setting_cache[center_node].append(setting_candidate)"""
            #for cached_settings in self.clean_setting_cache.values(): #ggggggg Start
                #for setting_candidate in cached_settings:
            #print("length of setting cache is", len(self.clean_setting_cache[center_node]))
            #settinglist = []
            #if self.cliques_with_epsilon:
            #    settingslist = [s for _, s in self.cliques_with_epsilon]
            self.cliques_with_epsilon = []
            self.incompletesettingcache = []
            #print("length of setting cache", len(self.clean_setting_cache))
            numincomplete = 0
            #for setting_candidate in self.clean_setting_cache[center_node]:
            for cached_settings in self.clean_setting_cache.values(): #ggggggg Start
                for setting_candidate in cached_settings:
                    working = setting_candidate.copy()
                    #print("checking candidate", working)
                    """if np.min(working) == 0:
                        if settinglist:
                            for o in settingslist:
                                if verbose:
                                    print("Checking",o)
                                if hit_by(o,working):
                                    non_id = o!=0
                                    working[non_id] = o[non_id]
                                if verbose:
                                    print("p =",working)
                                if np.min(working) > 0:
                                    print("completed setting using setting list")
                                    break"""
                    if np.min(working) == 0:
                        for idx in reversed(order):
                            o = self.obs[idx]
                            if verbose:
                                print("Checking",o)
                            if hit_by(o,working):
                                non_id = o!=0
                                working[non_id] = o[non_id]
                            if verbose:
                                print("p =",working)
                            if np.min(working) > 0:
                                #print("completed using shadow list")
                                self.incompletesettingcache.append(working)
                                numincomplete += 1
                                break
                    is_hit_candidate = []
                    delta = 0.02
                    is_hit_candidate = hit_by_batch_numba(self.obs , working)
                    #self.N_hits += is_hit_candidate
                    #self.cliques_with_epsilon.append((self.get_epsilon_Bernstein_no_restricted_validity_v2(delta), setting_candidate))
                    #self.N_hits -= is_hit_candidate
                    self.cliques_with_epsilon.append((self.total_hit_weight(weights, is_hit_candidate), working))
                    #print("length of cliques with epsilon is position 2", len(self.cliques_with_epsilon))
                    #delta = 0.33
            is_hit_candidate = []
            if not any((existing == shadowcliquesetting).all() for existing in self.clean_setting_cache[center_node]):
                    self.clean_setting_cache[center_node].append(shadowcliquesetting)
            is_hit_candidate = hit_by_batch_numba(self.obs , shadowcliquesetting)
            #self.N_hits += is_hit_candidate
            #self.cliques_with_epsilon.append((self.get_epsilon_Bernstein_no_restricted_validity_v2(delta), shadowcliquesetting))
            #self.N_hits -= is_hit_candidate
            self.cliques_with_epsilon.append((self.total_hit_weight(weights, is_hit_candidate), shadowcliquesetting))
            if not self.cliques_with_epsilon:
                raise RuntimeError("No cliques with valid observables were found. Likely due to representation mismatch or empty hit_cliques.")
            print("length of cliques with epsilon is", len(self.cliques_with_epsilon))
            print("number of incomplete settings is ", numincomplete)
            self.cliques_with_epsilon.sort(key=lambda x: x[0], reverse = True)
            _, best_clique = self.cliques_with_epsilon[0]
            self.selected_cliques.append(best_clique)
            if (len(best_clique) == len(shadowcliquesetting) and all(np.array_equal(a, b) for a, b in zip(best_clique, shadowcliquesetting))):
                self.shadow_was_best_count += 1
                print("Shadow clique was selected", self.shadow_was_best_count, "times")
            else:
                print("epsilon for best clique is",self.cliques_with_epsilon[0])
            setting = best_clique
            if not any((existing == best_clique).all() for existing in self.incompletesettingcache):
                self.incompletesettingselected += 1
                print("incomplete setting was selected",self.incompletesettingselected,"times")


In [ ]:
shadowcliquesetting = np.zeros(self.num_qubits,dtype=int)
            for idx in reversed(order):
                o = self.obs[idx]
                if verbose:
                    print("Checking",o)
                if hit_by(o,shadowcliquesetting):
                    non_id = o!=0
                    shadowcliquesetting[non_id] = o[non_id]
                if verbose:
                    print("p =",setting)
                if np.min(shadowcliquesetting) > 0:
                    break
            if verbose:
                print("Checking list of observables.")
            #cliques_with_epsilon = []
            delta = 0.02
            removedcliques = 0
            completecliques = 0
            valid_settings = []
            valid_cliques = []
            #start
            if center_node not in self.processed_center_node:  #gggggggg
                self.processed_center_node.append(center_node)
                hit_list = []
                hit_cliques = []
                non_id = first_obs != 0
                globalsetting[non_id] = first_obs[non_id]
                for o in self.obs:
                    if hit_by(o, globalsetting):
                        hit_list.append(o)
                if verbose:
                    print("First observable (setting):", first_obs)
                    print("Other observables hit by it:")
                    for ob in hit_list:
                        print(ob)
                if not hit_list:
                    raise RuntimeError("No hit list found.")
                hit_graph = build_hit_graph(hit_list)
                if not hit_graph:
                    raise RuntimeError("No hit graph found.")
                hit_cliques = find_cliques5(hit_graph)
                #print("length of hit cliques is", len(hit_cliques))
                self.clique_cache[center_node] = hit_cliques
                if not hit_cliques:
                    hit_cliques=[[center_node]]
                self.clean_setting_cache[center_node] = []
                self.clean_clique_cache[center_node] = []
                for clique in hit_cliques:
                    setting_candidate = np.zeros(self.num_qubits, dtype=int)
                    for o in clique:
                        o_arr = np.array(o)
                        if hit_by(o_arr, setting_candidate):
                            non_id = o_arr != 0
                            setting_candidate[non_id] = o_arr[non_id]
                        if np.min(setting_candidate) > 0:
                            break
                    """if np.min(setting_candidate) == 0:
                        for idx in reversed(order):
                            o = self.obs[idx]
                            if verbose:
                                print("Checking",o)
                            if hit_by(o,setting_candidate):
                                non_id = o!=0
                                setting_candidate[non_id] = o[non_id]
                            if verbose:
                                print("p =",setting_candidate)
                            if np.min(setting_candidate) > 0:
                                break"""
                    self.clean_clique_cache[center_node].append(clique)
                    if not any((existing == setting_candidate).all() for existing in self.clean_setting_cache[center_node]):
                        self.clean_setting_cache[center_node].append(setting_candidate)
            else:
                print("I already found cliques for this node")
                """else:
                    completecliques += 1
                    self.clean_clique_cache[center_node].append(clique)
                    if not any((existing == setting_candidate).all() for existing in self.clean_setting_cache[center_node]):
                        self.clean_setting_cache[center_node].append(setting_candidate)"""
            #for cached_settings in self.clean_setting_cache.values(): #ggggggg Start
                #for setting_candidate in cached_settings:
            #print("length of setting cache is", len(self.clean_setting_cache[center_node]))
            #settingslist = []
            #if self.cliques_with_epsilon:
            #    settingslist = [s for _, s in self.cliques_with_epsilon]
            #    print("length of settings list is", len(settingslist))
            self.cliques_with_epsilon = []
            self.incompletesettingcache = []
            #print("length of setting cache", len(self.clean_setting_cache))
            numincomplete = 0
            #numsetting = 0
            #idle = 0
            #for setting_candidate in self.clean_setting_cache[center_node]:
            for cached_settings in self.clean_setting_cache.values(): #ggggggg Start
                for setting_candidate in cached_settings:
                    working = setting_candidate.copy()
                    #print("checking candidate", working)
                    """if np.min(working) == 0:
                        if settingslist:
                            for o in settingslist:
                                if verbose:
                                    print("Checking",o)
                                if hit_by(o,working):
                                    non_id = o!=0
                                    working[non_id] = o[non_id]
                                if verbose:
                                    print("p =",working)
                                if np.min(working) > 0:
                                    #print("completed setting using setting list")
                                    numsetting += 1
                                    break"""

                    if np.min(working) == 0:
                        #id = (working == 0)
                        for idx in reversed(order):
                            o = self.obs[idx]
                            if verbose:
                                print("Checking",o)
                            if hit_by(o,working):
                                non_id = o!=0
                                working[non_id] = o[non_id]
                            if verbose:
                                print("p =",working)
                            if np.min(working) > 0:
                                #print("completed using shadow list")
                                self.incompletesettingcache.append(working)
                                numincomplete += 1
                                #idle += int(np.count_nonzero(id))
                                break
                    
                    is_hit_candidate = []
                    delta = 0.02
                    is_hit_candidate = hit_by_batch_numba(self.obs , working)
                    #self.N_hits += is_hit_candidate
                    #self.cliques_with_epsilon.append((self.get_epsilon_Bernstein_no_restricted_validity_v2(delta), setting_candidate))
                    #self.N_hits -= is_hit_candidate
                    self.cliques_with_epsilon.append((self.total_hit_weight(weights, is_hit_candidate), working))
                    #print("length of cliques with epsilon is position 2", len(self.cliques_with_epsilon))
                    #delta = 0.33
            #epsilon_shadow = []
            is_hit_candidate = []
            if not any((existing == shadowcliquesetting).all() for existing in self.clean_setting_cache[center_node]):
                    self.clean_setting_cache[center_node].append(shadowcliquesetting)
            is_hit_candidate = hit_by_batch_numba(self.obs , shadowcliquesetting)
            #self.N_hits += is_hit_candidate
            #self.cliques_with_epsilon.append((self.get_epsilon_Bernstein_no_restricted_validity_v2(delta), shadowcliquesetting))
            #self.N_hits -= is_hit_candidate
            self.cliques_with_epsilon.append((self.total_hit_weight(weights, is_hit_candidate), shadowcliquesetting))
            self.N_hits += is_hit_candidate
            epsilon_shadow = self.get_epsilon_Bernstein_no_restricted_validity_v2(delta)
            self.N_hits -= is_hit_candidate
            if not self.cliques_with_epsilon:
                raise RuntimeError("No cliques with valid observables were found. Likely due to representation mismatch or empty hit_cliques.")
            print("length of cliques with epsilon is", len(self.cliques_with_epsilon))
            #print("completed using shadow ", numincomplete)
            #print("completed using setting list ", numsetting)
            #print("size of idle part", idle)
            self.cliques_with_epsilon.sort(key=lambda x: x[0], reverse = True)
            _, best_clique = self.cliques_with_epsilon[0]
            self.selected_cliques.append(best_clique)
            if (len(best_clique) == len(shadowcliquesetting) and all(np.array_equal(a, b) for a, b in zip(best_clique, shadowcliquesetting))):
                self.shadow_was_best_count += 1
            else:
                print("epsilon for best clique is",self.cliques_with_epsilon[0])
            setting = best_clique
            if not any((existing == best_clique).all() for existing in self.incompletesettingcache):
                self.incompletesettingselected += 1
            print("incomplete setting was selected",self.incompletesettingselected,"times")
            print("Shadow clique was selected", self.shadow_was_best_count, "times")
            #epsilon_best = []
            is_hit_candidate = []
            is_hit_candidate = hit_by_batch_numba(self.obs , best_clique)
            self.N_hits += is_hit_candidate
            epsilon_best = self.get_epsilon_Bernstein_no_restricted_validity_v2(delta)
            self.N_hits -= is_hit_candidate